# Wind turbine project

This notebook is a brief example of the possibilities offered by the toolbox for modeling extreme values, adapted from the tools provided from the ResourceCode website.

It relies on the `pyextreme` library which get installed with the Resourcecode toolbox. Here we demonstrate 2 examples of univariate modeling as shown in class. For more information, see https://georgebv.github.io/pyextremes/.

# I. Wave Dynamics

## I.A. Characterizing the study site mean wave conditions

This notebook covers Part I.A of the wind turbine project: Characterizing the mean wave conditions and seasonal variability at the selected study site. The goal is to analyze long-term hindcast data to understand the typical and seasonal wave climate.

---

### Import Required Libraries




In [ ]:
import os
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
import pymannkendall as mk
from scipy import stats
from pathlib import Path
from pyextremes import (
    plot_mean_residual_life,
    plot_parameter_stability, 
    EVA
)
import resourcecode
from IPython import get_ipython

from resourcecode.eva import (
    get_fitted_models,
    get_gpd_parameters,
)
import warnings
warnings.filterwarnings("ignore")

os.makedirs('fig', exist_ok=True)
plt.savefig('fig/diagnostic_plot_bm.png', dpi=200, bbox_inches='tight')

# Enable inline plotting for Jupyter notebooks
get_ipython().run_line_magic("matplotlib", "inline")

---
### I.A.1. Mean Wave Conditions

Please download the variables corresponding to the significant wave height, mean wave period (Tm02), and mean wave direction. Then plot the time series of these variables during the 26-year period from 1994 to 2020, and calculate the mean significant wave height, period, and direction over the entire available time series.

### Chosen Study Site

**Site:** Bretagne Sud 1
**Coordinates:** $(47.3236111, -3.5522222)$. These were obtained by downloading the implementation zone KML file from https://www.eoliennesenmer.fr/facades-maritimes-en-france/facade-nord-atlantique-manche-ouest/projet-en-bretagne-sud/bretagne-sud-1 . We then went on Google Earth, picked a point at the centre of the implementation zone, and used those coordinates.

In [ ]:
client = resourcecode.Client()
# load the resourcecode dataset from Bretagne Sud 1
# find the closest point to the coordinates
lat = 47.3236111
long = -3.5522222
point_id, dist_m = resourcecode.data.get_closest_point(latitude=lat, longitude=long)
print(point_id, dist_m)

# get the data from the closest point
data = client.get_dataframe_from_criteria(
    """
{
    "node": 117231,
    "start": 0,
    "end": 99999903600,
    "parameter": ["hs","t02","dir","uwnd","vwnd","ucur","vcur","dpt"]
}
"""
)


---
### Data Source

The analysis will use long-term (1994-2020) hindcast simulations from the **ResourceCode wave database**.

**Variables to download**:
* Significant wave height ($H_{m0}$)
* Mean wave period ($T_{m02}$)
* Mean wave direction
* Wind velocity
* Current velocity





In [ ]:
# Derive only what ResourceCode does not already provide
# Assumes your download included: hs, t02, dir, uwnd, vwnd (and optionally ucur, vcur, dpt)

# Wind: speed (m/s) and coming-from direction (deg)
data["wspd"], data["wdir"] = resourcecode.utils.zmcomp2metconv(data["uwnd"], data["vwnd"])

# Currents (optional)
if {"ucur", "vcur"}.issubset(data.columns):
    data["cspd"], data["cdir"] = resourcecode.utils.zmcomp2metconv(data["ucur"], data["vcur"])

# Waves: use provided mean zero-crossing period
if "t02" in data.columns:
    data["Tm02"] = data["t02"]
else:
    raise KeyError("Missing 't02' in the request. Add 't02' to parameter list.")

# Keep dataset wave direction as-is; document convention once in the notebook.
# Do NOT overwrite 'dir' or try to recompute Tm02 from hs/dir.

data = data.sort_index()
data.head()


---
### Variables: definitions and units

**Core met-ocean time series**

| name   | meaning                                   | units |
|--------|-------------------------------------------|-------|
| `hs`   | significant wave height $H_{m0}=4\sqrt{m_0}$ | m     |
| `t02`  | mean zero-crossing period $T_{m02}=2\pi\sqrt{m_0/m_2}$ | s     |
| `dir`  | mean wave direction                       | °     |
| `spr`  | directional spreading                     | °     |
| `fp`   | spectral peak frequency                   | Hz    |
| `Tp`   | peak period $=1/fp$                     | s     |
| `uwnd` | eastward wind component                   | m·s⁻¹ |
| `vwnd` | northward wind component                  | m·s⁻¹ |
| `wspd` | wind speed $\sqrt{uwnd^2+vwnd^2}$       | m·s⁻¹ |
| `wdir` | wind direction                            | °     |
| `ucur` | eastward surface current                  | m·s⁻¹ |
| `vcur` | northward surface current                 | m·s⁻¹ |
| `cspd` | current speed $\sqrt{ucur^2+vcur^2}$    | m·s⁻¹ |
| `cdir` | current direction                         | °     |
| `dpt`  | water depth                               | m     |

**Spectral moments**

| name | meaning                               | units  |
|------|----------------------------------------|--------|
| `m0` | zeroth moment $\int S(\omega)\,d\omega$ | m²     |
| `m1` | first moment $\int \omega S(\omega)\,d\omega$ | m²·s⁻¹ |
| `m2` | second moment $\int \omega^2 S(\omega)\,d\omega$ | m²·s⁻² |

**Extreme value analysis (pyextremes)**

| item     | meaning                                  |
|----------|------------------------------------------|
| BM       | block-maxima extraction                  |
| POT      | peaks-over-threshold with declustering   |
| GEV $\mu,\sigma,\xi$ | location, scale, shape for BM     |
| GPD $\sigma,\xi$     | scale, shape for POT at a threshold |
| `r`      | min time separation between clusters     |
| `alpha`  | confidence level for intervals           |
| $z_T$  | return level for period $T$ years      |

**Direction conventions**

All directions are expressed **clockwise from North**.

| Variable | Convention | Notes |
|-----------|-------------|-------|
| `wdir` | *coming-from* | Derived from (`uwnd`, `vwnd`) using `resourcecode.utils.zmcomp2metconv`. |
| `dir_from` | *coming-from* | Use if dataset `dir` is *going-to*: convert by `dir_from = (dir + 180) % 360`. |
| `cdir_to` | *going-to* | Derived from (`ucur`, `vcur`); use as-is for current flow direction. |
| `cdir_from` | *coming-from* (optional) | For comparison with wave/wind directions, compute `cdir_from = (cdir_to + 180) % 360`. |

> Always state the convention in figure captions and keep it consistent across the notebook.

In [ ]:
# IA.1 — Mean wave conditions (1994–2020): time series, means, linear trends

def circmean_deg(x_deg):
    """Circular mean of degrees in [0, 360)."""
    x = pd.Series(x_deg).dropna().values
    if x.size == 0:
        return np.nan
    r = np.deg2rad(x)
    s = np.sin(r).sum()
    c = np.cos(r).sum()
    return (np.degrees(np.arctan2(s, c)) + 360.0) % 360.0

def verdict(p):
    return "significant" if p < 0.05 else "not significant"

# helper
def wrap360(a):
    return (a % 360.0 + 360.0) % 360.0

# Select analysis window
start = pd.Timestamp("1994-01-01")
end   = pd.Timestamp("2020-12-31 23:59:59")
needed = ["hs", "Tm02", "dir"]
missing = [v for v in needed if v not in data.columns]
if missing:
    raise KeyError(f"Missing variables for IA.1: {missing}")

df = data.loc[start:end, needed].copy()

# Monthly means (direction handled circularly)
monthly = pd.DataFrame({
    "hs":   df["hs"].resample("M").mean(),
    "Tm02": df["Tm02"].resample("M").mean(),
})
monthly["dir"] = df["dir"].resample("M").apply(circmean_deg)

# Overall means (direction: circular mean)
mean_hs   = df["hs"].mean()
mean_tm02 = df["Tm02"].mean()
mean_dir  = circmean_deg(df["dir"])

# Linear trends on monthly means
t_years = (monthly.index - monthly.index[0]).days / 365.2425

# Hs trend
hs_ok = monthly["hs"].dropna()
t_hs = t_years[hs_ok.index.get_indexer(hs_ok.index)]
hs_reg = stats.linregress(t_hs, hs_ok.values)
hs_slope_dec = hs_reg.slope * 10.0      # m per decade
hs_delta_30  = hs_reg.slope * 30.0      # m over 30 years

# Tm02 trend
tm_ok = monthly["Tm02"].dropna()
t_tm = t_years[tm_ok.index.get_indexer(tm_ok.index)]
tm_reg = stats.linregress(t_tm, tm_ok.values)
tm_slope_dec = tm_reg.slope * 10.0      # s per decade
tm_delta_30  = tm_reg.slope * 30.0      # s over 30 years

# Direction trend: unwrap, regress, report slope in deg/dec
dir_ok = monthly["dir"].dropna()
t_dir = t_years[dir_ok.index.get_indexer(dir_ok.index)]
dir_unwrap = np.degrees(np.unwrap(np.deg2rad(dir_ok.values)))
dir_reg = stats.linregress(t_dir, dir_unwrap)
dir_slope_dec = dir_reg.slope * 10.0    # deg per decade
dir_delta_30  = dir_reg.slope * 30.0    # deg over 30 years

# Plot monthly series (1994–2020)
fig, axes = plt.subplots(3, 1, figsize=(12, 9), sharex=True)

# Hs with fitted line
axes[0].plot(monthly.index, monthly["hs"], lw=0.8)
yhat_hs = hs_reg.intercept + hs_reg.slope * t_years
axes[0].plot(monthly.index, yhat_hs, lw=1.2)
axes[0].set_ylabel("Hs (m)")
axes[0].grid(alpha=0.3)
axes[0].set_title(f"Hs monthly mean | {hs_slope_dec:.3f} m/dec, p={hs_reg.pvalue:.3f} ({verdict(hs_reg.pvalue)})")

# Tm02 with fitted line
axes[1].plot(monthly.index, monthly["Tm02"], lw=0.8)
yhat_tm = tm_reg.intercept + tm_reg.slope * t_years
axes[1].plot(monthly.index, yhat_tm, lw=1.2)
axes[1].set_ylabel("Tm02 (s)")
axes[1].grid(alpha=0.3)
axes[1].set_title(f"Tm02 monthly mean | {tm_slope_dec:.3f} s/dec, p={tm_reg.pvalue:.3f} ({verdict(tm_reg.pvalue)})")

# Direction (circular monthly mean) with fitted line
axes[2].plot(monthly.index, monthly["dir"], lw=0.8)
yhat_dir_unwrap = dir_reg.intercept + dir_reg.slope * t_years
yhat_dir = wrap360(yhat_dir_unwrap)
axes[2].plot(monthly.index, yhat_dir, lw=1.2)  # add fit
axes[2].set_ylabel("Dir (deg, coming-from)")
axes[2].grid(alpha=0.3)
axes[2].set_title(
    f"Direction monthly circular mean | {dir_slope_dec:.2f}°/dec, "
    f"p={dir_reg.pvalue:.3f} ({verdict(dir_reg.pvalue)})"
)
axes[2].set_xlabel("Year")

plt.tight_layout()
plt.savefig("fig/IA1_mean_wave_conditions_timeseries.png", dpi=200, bbox_inches="tight")
plt.show()

# Printed summary for the report
print("IA.1 — Mean conditions over 1994–2020")
print(f"  Mean Hs   : {mean_hs:.3f} m")
print(f"  Mean Tm02 : {mean_tm02:.3f} s")
print(f"  Mean Dir  : {mean_dir:.1f}° (coming-from)")

print("\nLinear trends on monthly means (least squares):")
print(f"  Hs   : {hs_slope_dec:.3f} m/dec  (p={hs_reg.pvalue:.3f}, n={hs_ok.size}), Δ30y={hs_delta_30:.3f} m")
print(f"  Tm02 : {tm_slope_dec:.3f} s/dec  (p={tm_reg.pvalue:.3f}, n={tm_ok.size}), Δ30y={tm_delta_30:.3f} s")
print(f"  Dir  : {dir_slope_dec:.2f} °/dec (p={dir_reg.pvalue:.3f}, n={dir_ok.size}), Δ30y={dir_delta_30:.2f} °")

print("\nInterpretation rule-of-thumb: treat p<0.05 as evidence of a trend. Use Δ30y to state expected change over a turbine lifetime.")

In [ ]:
result_hs  = mk.original_test(hs_ok.values, alpha=0.05)
result_period = mk.original_test(tm_ok.values, alpha=0.050)
result_direction = mk.original_test(dir_ok.values, alpha=0.05)

print("\nResults for Significant Wave Height (Hs):")
print(f"  Trend: {result_hs.trend}")
print(f"  H (Test Statistic): {result_hs.h}")
print(f"  P-value: {result_hs.p:.4f}")
print(f"  Z-Score: {result_hs.z:.4f}")
print(f"  Tau: {result_hs.Tau:.4f}")
print(f"  Sen's Slope: {result_hs.slope:.4f}")
print(f"  Intercept: {result_hs.intercept:.4f}")

print("\nResults for Wave Period (Tm02):")
print(f"  Trend: {result_period.trend}")
print(f"  H (Test Statistic): {result_period.h}")
print(f"  P-value: {result_period.p:.4f}")
print(f"  Z-Score: {result_period.z:.4f}")
print(f"  Tau: {result_period.Tau:.4f}")
print(f"  Sen's Slope: {result_period.slope:.4f}")
print(f"  Intercept: {result_period.intercept:.4f}")

print("\nResults for Wave Direction:")
print(f"  Trend: {result_direction.trend}")
print(f"  H (Test Statistic): {result_direction.h}")
print(f"  P-value: {result_direction.p:.4f}")
print(f"  Z-Score: {result_direction.z:.4f}")
print(f"  Tau: {result_direction.Tau:.4f}")
print(f"  Sen's Slope: {result_direction.slope:.4f}")
print(f"  Intercept: {result_direction.intercept:.4f}")

**Q: Do any trends in the wave height, period, or direction exist over the 26 year time period? Do you expect there to be changes in the mean conditions during the 30-year lifetime of the wind turbine? If so, why?**

**Answer:** No statistically significant trends were detected in wave height ($H_s$), period ($T_{m02}$), or direction using either Ordinary Least Squares (OLS) or the Mann-Kendall (MK) test on monthly mean data. Expected 30-year changes in mean conditions remain negligible relative to natural variability.

---
### Mean Conditions and Trend Analysis (1994-2020)

**Mean conditions**
-   $\overline{H_s} = 2.092\ \text{m}$
-   $\overline{T_{m02}} = 5.785\ \text{s}$
-   Mean direction $= 275.5^\circ$ (coming from W)

---
**Trend Analysis Methods**
-   Aggregate hourly data to monthly means to reduce noise while retaining seasonal information influence.
-   **Method 1: Ordinary Least Squares (OLS)**: Perform regression vs time in years. Direction handled with circular monthly mean, then unwrapped.
-   **Method 2: Mann-Kendall (MK) Test**: Apply the non-parametric test to detect monotonic trends in the monthly mean time series.
-   Significance for both methods assessed at $\alpha=0.05$.

---
**OLS Results** (slope per decade; $\Delta 30\text{y}$ is implied 30-year change; $n=324$ monthly means)
-   $H_s$: $+0.015\ \text{m/dec}$, $p=0.776$, $\Delta 30\text{y}\approx +0.046\ \text{m}$. (No significant trend)
-   $T_{m02}$: $+0.086\ \text{s/dec}$, $p=0.150$, $\Delta 30\text{y}\approx +0.259\ \text{s}$. (No significant trend)
-   Direction: $+0.75^\circ/\text{dec}$, $p=0.349$, $\Delta 30\text{y}\approx +2.26^\circ$. (No significant trend)

---
**MK Results (on monthly means)**
-   $H_s$: $p=0.8098$ (no significant trend); Sen's slope $\approx +1.0\times10^{-4}$ m/month ($\approx +0.012$ m/decade).
-   $T_{m02}$: $p=0.1957$ (no significant trend); Sen's slope $\approx +7.0\times10^{-4}$ s/month ($\approx +0.084$ s/decade).
-   Direction: $p=0.4405$ (no significant trend); Sen's slope $\approx +5.5\times10^{-3}$ deg/month ($\approx +0.66^\circ$/decade).

*(Note: Sen's slopes converted approximately from per-month to per-decade for comparison with OLS results)*

---
**Interpretation**
-   Both OLS and Mann-Kendall analyses performed on monthly mean data consistently indicate no statistically significant secular trends in mean $H_s$, $T_{m02}$, or direction over the 1994-2020 period.
-   The magnitudes of the calculated slopes (both OLS and Sen's slope) are very small, suggesting that any potential underlying linear or monotonic change over the 26 years is minimal compared to the observed variability.
-   Expected 30-year changes based on these negligible trends are small compared to the substantial seasonal and interannual variability present in the data.
-   For design purposes over the next 30 years, emphasis should likely remain on characterizing the existing variability (seasonal, interannual) and extreme conditions rather than adjusting significantly for potential drifts in mean conditions.

---
*Figure:* `fig/IA1_mean_wave_conditions_timeseries.png`.



---
### I.A.2. Most Common Wave Conditions

To identify the most common operating conditions, we will create a 2D histogram (scatter diagram) of significant wave height ($H_{m0}$) versus mean wave period ($T_{m02}$). We will also plot a wave rose to identify the most common wave incidence direction(s).









In [ ]:
plt.figure(figsize=(10,8))
plt.hist2d(data['hs'], data['t02'], bins=100, cmap='plasma', density=True)
plt.colorbar(label='Density')
plt.xlabel('Significant Wave Height $H_s$ [m]')
plt.ylabel('Mean Wave Period $T_{m02}$ [s]')
plt.title('Wave Height vs Period Density Plot')
plt.savefig('graphs/wave_height_period_density.png')
plt.show()
plt.close()

**Q: What is the water depth at this location? Indicate on the histrogram for what wave conditions the waves are considered deep water waves? linear waves?**

In [ ]:
print(data['dpt'].std())

mean_water_depth = data['dpt'].mean()
print(f'mean water depth at location (lat, long) = ({lat}, {long}) is {mean_water_depth:.2f} m')

# Calculating wave length using the linear wave theory
# wave length = g * T^2 / (2 * pi)
g = 9.81 # m/s^2
wave_length = g * (data['t02']**2) / (2 * np.pi)
water_depth = data['dpt']

# check condition for deep water waves
deep_water_length_condition = 2 * mean_water_depth
transition_water_length_condition = 25 * mean_water_depth

deep_water_condition = wave_length < deep_water_length_condition
# calculate the percentage of deep water waves
deep_water_percentage = deep_water_condition.mean()
print(f'percentage of deep water waves: {deep_water_percentage:.2%}')

# Compute corresponding period for deep water waves
deep_water_period_condition = 4 * np.sqrt(mean_water_depth / g)
transition_water_period_condition = 25 * np.sqrt(mean_water_depth / g)

print(f'deep water period: {deep_water_period_condition:.2f} s')
# check condition for linear waves
#linear_waves_condition = wave_length > 20 * mean_water_depth

### plot the wave length distribution ###
plt.figure(figsize=(10, 6))

# Corrected plt.hist call: remove wave_length.count
plt.hist(wave_length, bins=50, density=True, alpha=0.7, label='Wave Length Distribution')

# Add the threshold line (assuming L < 2h for deep water)
plt.axvline(x=deep_water_length_condition, color='r', linestyle='--',
            label=f'Deep Water Threshold (L < {deep_water_length_condition:.1f}m)')
plt.axvline(x=transition_water_length_condition, color='g', linestyle='--',
            label=f'Upper Transition Threshold to Linear Waves (L < {transition_water_length_condition:.1f}m)')
plt.xlabel('Wave Length (m)')
plt.ylabel('Density') # Changed label to Density since density=True
plt.title('Wave Length Distribution')
plt.legend()
plt.grid(True)
plt.show()
plt.close() # Close after showing

plt.figure(figsize=(10, 6))
### plot the wave period distribution ###
plt.hist(data['t02'], bins=50, density=True, alpha=0.7, label='Wave Period Distribution')

# Add the threshold line (assuming L < 2h for deep water)
plt.axvline(x=deep_water_period_condition, color='r', linestyle='--',
            label=f'Deep Water Threshold (T < {deep_water_period_condition:.1f}s)')
plt.axvline(x=transition_water_period_condition, color='g', linestyle='--',
            label=f'Transition to Linear Waves (T < {transition_water_period_condition:.1f}s)')
plt.xlabel('Wave Period (s)')
plt.xscale('log')
plt.ylabel('Density') # Changed label to Density since density=True
plt.title('Wave Period Distribution')
plt.legend()
plt.grid(True)
plt.show()
plt.close()

# --- 6. Result Summary ---
print("\n--- Result Summary ---")
print(f"Analysis based on Mean Depth h = {mean_water_depth:.2f} m")

print("\nThresholds:")
print(f"  Deep Water:     L < {deep_water_length_condition:.1f} m   |   T < {deep_water_period_condition:.2f} s")
print(f"  Intersection of Deep and Shallow Water:  L > {transition_water_length_condition:.1f} m  |   T > {transition_water_period_condition:.2f} s")

# Compute the percentage of deep water waves based on the approximate wavelength
is_deep_water_wave = wave_length < deep_water_length_condition
is_transition_zone_wave = (
    (deep_water_length_condition < wave_length) &
    (wave_length < transition_water_length_condition)
)

is_deep_water_period = data['t02'] < deep_water_period_condition
is_transition_zone_period = (
    (deep_water_period_condition < data['t02']) &
    (data['t02'] < transition_water_period_condition)
)
# get a percentage of the boolean series of true and false
deep_water_percentage_L_approx = is_deep_water_wave.mean()
transition_percentage_L_approx = is_transition_zone_wave.mean()
deep_water_percentage_T_approx = is_deep_water_period.mean()
transition_percentage_T_approx = is_transition_zone_period.mean()

print("\nClassification based on Approximate Wavelength (L_approx = gT² / 2π):")
print(f"  Percentage Deep Water Wave Lenghts: {deep_water_percentage_L_approx:.2%}")
print(f"  Percentage Transition Zone Wave Lenghts: {transition_percentage_L_approx:.2%}")

# Characterizing the transition band for T
print("\nClassification based on Approximate Wavelength (L_approx = gT² / 2π):")
print(f"  Percentage Deep Water Wave Periods: {deep_water_percentage_T_approx:.2%}")
print(f"  Percentage Transition Zone Wave Periods: {transition_percentage_T_approx:.2%}")

### Wave Regime Classification

**Method**
-   The wave conditions represented by the hourly significant wave height ($H_s$) and mean period ($T_{m02}$) were classified into deep, transitional, and shallow water regimes.
-   Classification was based on standard non-dimensional criteria involving water depth ($h$), wavelength ($L$), and period ($T$).
-   A **mean water depth of $h = 31.50$ m** was used for calculating the thresholds. This simplification is justified by the low standard deviation observed in the depth data ($\sigma_h = 1.2$ m), indicating relatively consistent depth at the site.
-   Wavelength ($L_{approx}$) was estimated using the deep water approximation $L_{approx} = gT_{m02}^2 / (2\pi)$ for initial classification.
-   Classification was also performed directly using the period $T_{m02}$.

---
**Thresholds** (based on $h = 31.50$ m)
-   **Deep Water:** $h/L > 1/2 \implies L < 63.0$ m; or $T\sqrt{g/h} < 4 \implies T < 7.17$ s.
-   **Shallow Water:** $h/L < 1/25 \implies L > 787.6$ m; or $T\sqrt{g/h} > 25 \implies T > 44.80$ s.
-   **Transition Zone:** Conditions falling between the deep and shallow water limits.

---
**Results**
-   Using the **period-based classification ($T_{m02}$)**:
    -   Deep Water ($T < 7.17$ s): **93.71%**
    -   Transition Zone ($7.17 \text{ s} \le T \le 44.81$ s): **6.29%**
    -   Shallow Water ($T > 44.81$ s): **0.00%**
-   Using the **approximate wavelength classification ($L_{approx}$)**:
    -   Deep Water ($L_{approx} < 63.0$ m): **86.70%**
    -   Transition Zone ($63.0 \text{ m} \le L_{approx} \le 630.0$ m): **13.30%**
    -   Shallow Water ($L_{approx} > 630.0$ m): **0.00%**

---
**Interpretation**
-   The vast majority of wave conditions at this site fall within the **deep water regime** based on both period and approximate wavelength classifications, although the period-based method indicates a higher percentage.
-   A smaller fraction (6-13%) operates within the **transition zone**.
-   **Shallow water conditions are negligible** according to these criteria and the dataset.
-   The difference between the period and approximate wavelength classifications highlights the limitation of using the deep water formula ($L_{approx}$) for all conditions; the period-based classification using $T\sqrt{g/h}$ is more direct.


---
### I.A.3. Seasonal Variability

Next, we will evaluate temporal variability by calculating and plotting the mean wave height, period, and direction as a function of the month of the year (e.g., mean for January, February, etc.).

**Data and method.** Hourly hindcast at coordinates. Monthly climatology across all years.  
- Height and period: arithmetic mean with interquartile range (IQR).  
- Direction: circular mean with circular standard deviation, expressed as *coming-from* degrees.  
- Angles unwrapped around the overall mean for continuity.

In [ ]:
# --- IA.3 Seasonal variability: monthly climatology of Hs, Tm02, and direction (mean ± IQR) ---

# inputs
need  = {"hs", "Tm02", "dir"}
if not need.issubset(data.columns):
    raise KeyError(f"IA.3 needs columns {sorted(need)} in `data`.")

df3 = data.loc[start:end, ["hs", "Tm02", "dir"]].copy()
if not isinstance(df3.index, pd.DatetimeIndex):
    raise TypeError("`data` index must be a DatetimeIndex.")

# group by calendar month across all years
g = df3.groupby(df3.index.month)  # 1..12

# numeric climatology for Hs and Tm02
hs_mean = g["hs"].mean()
hs_q25  = g["hs"].quantile(0.25)
hs_q75  = g["hs"].quantile(0.75)

tm_mean = g["Tm02"].mean()
tm_q25  = g["Tm02"].quantile(0.25)
tm_q75  = g["Tm02"].quantile(0.75)

n_samples = g.size()

# circular climatology for direction (vectorised; stable across pandas versions)
dd = df3[["dir"]].copy()
m = dd["dir"].notna()
dd.loc[m, "sin"] = np.sin(np.deg2rad(dd.loc[m, "dir"]))
dd.loc[m, "cos"] = np.cos(np.deg2rad(dd.loc[m, "dir"]))

dg = dd.groupby(dd.index.month)
sin_mean = dg["sin"].mean()
cos_mean = dg["cos"].mean()
n_dir    = dg["dir"].count()

R = np.hypot(sin_mean, cos_mean)                                  # mean resultant length
dir_mean = (np.degrees(np.arctan2(sin_mean, cos_mean)) + 360) % 360
dir_cstd = np.degrees(np.sqrt(np.maximum(0.0, -2.0 * np.log(np.clip(R, 1e-12, 1.0)))))  # circular std (deg)

# unwrap monthly direction around overall circular mean to avoid 0/360 jump
sin_all = np.sin(np.deg2rad(df3["dir"].dropna())).mean()
cos_all = np.cos(np.deg2rad(df3["dir"].dropna())).mean()
overall_dir = (np.degrees(np.arctan2(sin_all, cos_all)) + 360) % 360
dir_mean_unwrapped = overall_dir + ((dir_mean - overall_dir + 540.0) % 360.0 - 180.0)

# assemble single DataFrame and ensure months 1..12 exist and in order
clim = pd.DataFrame({
    "hs_mean": hs_mean,
    "hs_q25":  hs_q25,
    "hs_q75":  hs_q75,
    "Tm02_mean": tm_mean,
    "Tm02_q25":  tm_q25,
    "Tm02_q75":  tm_q75,
    "N_samples": n_samples,
    "dir_mean": dir_mean,
    "R": R,
    "dir_cstd": dir_cstd,
    "n_dir": n_dir,
    "dir_mean_unwrapped": dir_mean_unwrapped,
}).reindex(range(1, 13))
clim.index.name = "month"
clim["month_name"] = pd.to_datetime(clim.index, format="%m").month_name().str.slice(0, 3)

# save numeric table
clim.to_csv("fig/IA3_seasonal_climatology.csv", float_format="%.3f")

# plot
fig, axes = plt.subplots(3, 1, figsize=(12, 9), sharex=True)
x = np.arange(1, 13)
labels = clim["month_name"].values

# Hs: mean ± IQR
ax = axes[0]
ax.plot(x, clim["hs_mean"].values, lw=1.8)
ax.fill_between(x, clim["hs_q25"].values, clim["hs_q75"].values, alpha=0.25)
ax.set_ylabel("Hs (m)")
ax.grid(alpha=0.3)
ax.set_title("Monthly climatology: Hs (mean ± IQR)")

# Tm02: mean ± IQR
ax = axes[1]
ax.plot(x, clim["Tm02_mean"].values, lw=1.8)
ax.fill_between(x, clim["Tm02_q25"].values, clim["Tm02_q75"].values, alpha=0.25)
ax.set_ylabel("Tm02 (s)")
ax.grid(alpha=0.3)
ax.set_title("Monthly climatology: Tm02 (mean ± IQR)")

# Direction: circular mean ± circular std (unwrapped for continuity)
ax = axes[2]
y = clim["dir_mean_unwrapped"].values
yerr = clim["dir_cstd"].values
ax.errorbar(x, y, yerr=yerr, fmt="-o", lw=1.5, capsize=3)
pad = 5.0
ymin, ymax = np.nanmin(y - yerr), np.nanmax(y + yerr)
ax.set_ylim(ymin - pad, ymax + pad)
ax.set_ylabel("Direction (deg, coming-from)")
ax.grid(alpha=0.3)
ax.set_title("Monthly climatology: wave direction (circular mean ± circular std)")
ax.set_xticks(x)
ax.set_xticklabels(labels)

plt.tight_layout()
plt.savefig("fig/IA3_seasonal_climatology.png", dpi=200, bbox_inches="tight")
plt.show()

# quick textual summary
hs_max_mo = int(clim["hs_mean"].idxmax())
hs_min_mo = int(clim["hs_mean"].idxmin())
tm_max_mo = int(clim["Tm02_mean"].idxmax())
tm_min_mo = int(clim["Tm02_mean"].idxmin())
print("IA.3 — Seasonal climatology (1994–2020)")
print(f"  Hs peaks in {pd.to_datetime(hs_max_mo, format='%m').month_name()} "
      f"and is lowest in {pd.to_datetime(hs_min_mo, format='%m').month_name()}.")
print(f"  Tm02 peaks in {pd.to_datetime(tm_max_mo, format='%m').month_name()} "
      f"and is lowest in {pd.to_datetime(tm_min_mo, format='%m').month_name()}.")
print(f"  Direction mean (overall) ≈ {overall_dir:.1f}° (coming-from).")
print("  Saved: fig/IA3_seasonal_climatology.png and fig/IA3_seasonal_climatology.csv")


**Q: Do you observe any seasonal trends in wave height, period, or direction? If so, why?**


### Results

**Significant wave height, $H_s$.** Clear winter maximum and summer minimum. IQR widens in winter and narrows in summer.  
**Interpretation.** Boreal-winter extratropical cyclones increase wind speed and fetch over the open North Atlantic, augmenting wind-sea and incoming swell towards south Brittany. Summer conditions are dominated by weaker winds and shorter fetch, so mean $H_s$ decreases and variability contracts. Because wave energy density scales as
$$
E=\tfrac{1}{8}\,\rho g\,H_s^2,
$$
higher winter $H_s$ implies markedly greater wave energy arriving at the site.

**Mean zero-crossing period, $T_{m02}$.** Co-varies with $H_s$: longer in winter, shorter in summer.  
**Interpretation.** Winter storms generate longer-period swell over long fetches. Deep-water dispersion $\big(L\approx gT^2/2\pi,\ c_g \approx gT/4\pi\big)$ favours the far-field propagation of longer-period energy into the Bay of Biscay. In summer, local wind-sea contribution increases and typical periods shorten.

### Wave direction (coming-from, clockwise from North)

**Convention.** $0^\circ=\mathrm{N},\ 90^\circ=\mathrm{E},\ 180^\circ=\mathrm{S},\ 270^\circ=\mathrm{W}$.

**Seasonal pattern.** The monthly circular mean increases from winter ($\sim 240^\circ$) to summer ($\sim 255^\circ$), then decreases again in autumn.

**Interpretation** Larger angles imply a more westerly approach. Thus, waves are slightly more **W–WSW** in summer and shift a little toward **SW** in winter.

**Regional context.** Along the southern Brittany–Bay of Biscay sector, the summer expansion of the Azores High favours a more zonal (westerly) approach at the shelf break. In winter, frequent lows entering the Bay introduce a modest southerly component in the incident swell, yielding the observed decrease in direction angle.

**Dispersion note.** The vertical bars are circular standard deviations (spread of hourly directions), not standard errors. They are of similar magnitude across months, so the dataset does **not** support a strong seasonal change in directional spread.

## Implications

- **Operations.** Highest loads and sea states occur in winter months; scheduling for installation or maintenance is more feasible in late spring–summer.  
- **Variability.** Broad winter IQRs indicate stronger interannual modulation of sea states; design and planning should not rely on a single “typical” winter value.  
- **Directionality.** The prevailing W–WSW approach is stable enough to justify directional binning around that sector for further analyses and for extreme value modelling in Part II.

## Notes and caveats

- Monthly means smooth synoptic extremes; use EVA for design loads.  
- Direction statistics use circular metrics; reported means reflect modal approach rather than arithmetic averages near $0/360^\circ$.  
- The site is predominantly deep water, so seasonal patterns reflect atmospheric forcing and basin geometry rather than depth-limited effects.


---
### I.A.4. Mean Wind and Current Conditions

We will also evaluate the wind and current conditions by plotting rose diagrams of their respective velocities and directions.


In [ ]:
# --- IA.4 Mean wind and current conditions: roses + means (overall, seasonal) ---
# -----------------------
# Config
# -----------------------
# Calm thresholds used for ROSES only (kept in means table for reference)
CALM_WIND = 0.5    # m/s
CALM_CURR = 0.05   # m/s

# Speed bins
# Wind: [0.5–2), 2–4, 4–6, …, ≥16 m/s
WIND_BINS = np.r_[0.5, np.arange(2, 18, 2), np.inf]
# Currents: [0.05–0.1), 0.1–0.2, 0.2–0.3, 0.3–0.5, ≥0.5 m/s
CURR_BINS = np.array([0.05, 0.10, 0.15, 0.2, np.inf])

# Direction sectors
WIND_SECTORS = 16   # 22.5°
CURR_SECTORS = 36   # 10°

# Date window to match the project
# -----------------------
# Helpers
# -----------------------
def season_label(month):
    # DJF, MAM, JJA, SON
    if month in (12, 1, 2):
        return "DJF"
    if month in (3, 4, 5):
        return "MAM"
    if month in (6, 7, 8):
        return "JJA"
    return "SON"

def rose_counts(dir_deg, spd, spd_bins, sectors=16, calm_thresh=0.0):
    """
    Return stacked counts per direction sector and speed bin, plus metadata.
    Directions: degrees, 0°=N, clockwise positive.
    Excludes values with spd <= calm_thresh.
    """
    mask = np.isfinite(dir_deg) & np.isfinite(spd) & (spd > calm_thresh)
    d = np.asarray(dir_deg)[mask] % 360.0
    s = np.asarray(spd)[mask]

    if d.size == 0:
        counts = np.zeros((sectors, len(spd_bins)-1), dtype=int)
        return counts, np.array([]), 0

    width = 360.0 / sectors
    sector_idx = np.floor(d / width).astype(int)
    sector_idx[sector_idx == sectors] = sectors - 1

    bin_idx = np.digitize(s, spd_bins) - 1
    bin_idx = np.clip(bin_idx, 0, len(spd_bins)-2)

    counts = np.zeros((sectors, len(spd_bins)-1), dtype=int)
    for k in range(d.size):
        counts[sector_idx[k], bin_idx[k]] += 1

    return counts, mask, d.size  # d.size = included samples after calm filter

def plot_rose(dir_deg, spd, spd_bins, fname, title, calm_thresh=0.0, sectors=16):
    """
    Polar stacked-bar rose. Heights are percentages of included samples.
    Direction convention: 0° at North, clockwise positive.
    """
    counts, mask, n_used = rose_counts(dir_deg, spd, spd_bins, sectors, calm_thresh)
    if n_used == 0:
        print(f"{title}: no data above calm threshold.")
        return

    sector_totals = counts.sum(axis=1).astype(float)
    sector_totals[sector_totals == 0] = 1.0
    frac = counts / sector_totals[:, None]
    pct_sector = 100.0 * counts.sum(axis=1) / n_used

    theta = np.deg2rad(np.arange(0, 360, 360/sectors))
    width = 2.0 * np.pi / sectors

    plt.figure(figsize=(8, 8))
    ax = plt.subplot(111, polar=True)
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)

    bottoms = np.zeros(sectors)
    labels = []
    for j in range(len(spd_bins)-1):
        label = f"{spd_bins[j]:g}–{spd_bins[j+1]:g} m/s" if np.isfinite(spd_bins[j+1]) else f"{spd_bins[j]:g}+ m/s"
        labels.append(label)
        heights = pct_sector * frac[:, j]
        ax.bar(theta, heights, width=width, bottom=bottoms, align="edge",
               edgecolor="black", linewidth=0.3)
        bottoms += heights

    ax.set_rlabel_position(225)
    rmax = max(5.0, np.ceil(bottoms.max() / 5.0) * 5.0)
    ax.set_ylim(0, rmax)
    ax.set_yticks(np.linspace(0, rmax, 5))
    ax.set_yticklabels([f"{v:.0f}%" for v in np.linspace(0, rmax, 5)])

    ax.set_title(title + f"\n(calms ≤ {calm_thresh} m/s excluded; n={n_used})",
                 va="bottom", fontsize=11)
    ax.legend(labels, loc="lower left", bbox_to_anchor=(0.9, -0.02),
              frameon=True, fontsize=9)

    plt.tight_layout()
    plt.savefig(fname, dpi=200, bbox_inches="tight")
    plt.show()
    plt.close()

def mean_table(dir_deg, spd, label, calm_thresh=None):
    """
    Mean speed (arithmetic) and mean direction (circular) overall and by season.
    Directions in degrees [0, 360).
    calm_thresh listed for reference; does not alter means.
    """
    idx = data.loc[start:end].index
    df = pd.DataFrame({"spd": spd.loc[idx], "dir": dir_deg.loc[idx]})
    df["season"] = df.index.month.map(season_label)

    def agg(g):
        return pd.Series({
            "mean_speed_mps": g["spd"].mean(),
            "mean_dir_deg": circmean_deg(g["dir"]),
            "n": g["dir"].count()
        })

    overall = agg(df)
    by_season = df.groupby("season", sort=False).apply(agg)

    overall.name = label
    out = {"overall": overall.to_frame().T, "season": by_season}
    out["meta"] = {"calm_threshold_listed_only_mps": calm_thresh}
    return out

# -----------------------
# Prepare series and conventions
# -----------------------
# Wind: already "coming-from" via zmcomp2metconv
wspd = data["wspd"].copy()
wdir_from = data["wdir"].copy()  # coming-from, 0°=N, clockwise

# Currents: convert to "going-to" for the rose
if {"cspd", "cdir"}.issubset(data.columns):
    cspd = data["cspd"].copy()
    cdir_to = (data["cdir"] + 180.0) % 360.0
else:
    cspd = None
    cdir_to = None

# Restrict to analysis window
sel = slice(start, end)
wspd = wspd.loc[sel]
wdir_from = wdir_from.loc[sel]
if cspd is not None:
    cspd = cspd.loc[sel]
    cdir_to = cdir_to.loc[sel]

# -----------------------
# 1) Full-period roses
# -----------------------
plot_rose(
    dir_deg=wdir_from, spd=wspd, spd_bins=WIND_BINS,
    fname="fig/IA4_wind_rose.png",
    title="Wind rose (coming-from; 0°=N, clockwise)",
    calm_thresh=CALM_WIND, sectors=WIND_SECTORS
)

if cspd is not None:
    plot_rose(
        dir_deg=cdir_to, spd=cspd, spd_bins=CURR_BINS,
        fname="fig/IA4_current_rose.png",
        title="Current rose (going-to; 0°=N, clockwise)",
        calm_thresh=CALM_CURR, sectors=CURR_SECTORS
    )

# -----------------------
# 2) Seasonal roses (DJF, MAM, JJA, SON)
# -----------------------
def plot_rose_seasons(dir_series, spd_series, spd_bins, calm_thresh, sectors, title_base, fname):
    seasons = ["DJF", "MAM", "JJA", "SON"]
    fig, axes = plt.subplots(2, 2, figsize=(12, 10), subplot_kw=dict(polar=True))
    axes = axes.ravel()
    for i, s in enumerate(seasons):
        sel = dir_series.index.map(lambda t: season_label(t.month) == s)
        d = dir_series[sel]
        v = spd_series[sel]
        counts, mask, n_used = rose_counts(d.values, v.values, spd_bins, sectors, calm_thresh)
        theta = np.deg2rad(np.arange(0, 360, 360/sectors))
        width = 2.0 * np.pi / sectors

        ax = axes[i]
        ax.set_theta_zero_location("N")
        ax.set_theta_direction(-1)

        if n_used == 0:
            ax.set_title(f"{s} (n=0)")
            continue

        sector_totals = counts.sum(axis=1).astype(float)
        sector_totals[sector_totals == 0] = 1.0
        frac = counts / sector_totals[:, None]
        pct_sector = 100.0 * counts.sum(axis=1) / n_used

        bottoms = np.zeros(sectors)
        for j in range(len(spd_bins)-1):
            heights = pct_sector * frac[:, j]
            ax.bar(theta, heights, width=width, bottom=bottoms, align="edge",
                   edgecolor="black", linewidth=0.3)
            bottoms += heights

        ax.set_rlabel_position(225)
        rmax = max(5.0, np.ceil(bottoms.max() / 5.0) * 5.0)
        ax.set_ylim(0, rmax)
        ax.set_yticks(np.linspace(0, rmax, 5))
        ax.set_yticklabels([f"{v:.0f}%" for v in np.linspace(0, rmax, 5)])
        ax.set_title(f"{s} (calms ≤ {calm_thresh} m/s; n={n_used})", fontsize=10)

    fig.suptitle(title_base, y=0.98)
    plt.tight_layout(rect=[0, 0.02, 1, 0.95])
    plt.savefig(fname, dpi=200, bbox_inches="tight")
    plt.show()
    plt.close()

plot_rose_seasons(
    wdir_from, wspd, WIND_BINS, CALM_WIND, WIND_SECTORS,
    "Wind roses by season (coming-from; 0°=N, clockwise)",
    "fig/IA4_wind_rose_seasons.png"
)

if cspd is not None:
    plot_rose_seasons(
        cdir_to, cspd, CURR_BINS, CALM_CURR, CURR_SECTORS,
        "Current roses by season (going-to; 0°=N, clockwise)",
        "fig/IA4_current_rose_seasons.png"
    )

# -----------------------
# 3) Mean conditions tables and CSV
# -----------------------
wind_stats = mean_table(wdir_from, wspd, label="wind", calm_thresh=CALM_WIND)
if cspd is not None:
    curr_stats = mean_table(cdir_to, cspd, label="current", calm_thresh=CALM_CURR)

frames = []
wind_overall = wind_stats["overall"].assign(kind="wind")
wind_season  = wind_stats["season"].assign(kind="wind", level="season").reset_index().rename(columns={"season":"group"})
frames += [wind_overall.assign(level="overall", group="all"), wind_season]

if cspd is not None:
    curr_overall = curr_stats["overall"].assign(kind="current")
    curr_season  = curr_stats["season"].assign(kind="current", level="season").reset_index().rename(columns={"season":"group"})
    frames += [curr_overall.assign(level="overall", group="all"), curr_season]

stats_df = pd.concat(frames, ignore_index=True)
stats_df = stats_df[["kind", "level", "group", "mean_speed_mps", "mean_dir_deg", "n"]]
stats_df.to_csv("fig/IA4_mean_wind_current_stats.csv", index=False, float_format="%.3f")

# Print seasonal values for wind and current

season_order = CategoricalDtype(["DJF", "MAM", "JJA", "SON"], ordered=True)

def print_seasonals(kind_label, dir_note):
    df = stats_df.query("kind == @kind_label and level == 'season'").copy()
    df["group"] = df["group"].astype(season_order)
    df = df.sort_values("group")
    print(f"\nIA.4 — Seasonal means, {kind_label} ({dir_note})")
    print(df[["group", "mean_speed_mps", "mean_dir_deg", "n"]]
          .rename(columns={"group": "season"})
          .to_string(index=False,
                     formatters={
                         "mean_speed_mps": lambda v: f"{v:.3f}",
                         "mean_dir_deg":  lambda v: f"{v:.1f}",
                         "n":             lambda v: f"{int(v)}"
                     }))

print_seasonals("wind", "coming-from")
print_seasonals("current", "going-to")

print("IA.4 — Mean conditions (1994–2020)")
print(stats_df.query("level == 'overall'"))
print("\nSaved figures:")
print("  fig/IA4_wind_rose.png")
if cspd is not None:
    print("  fig/IA4_current_rose.png")
print("  fig/IA4_wind_rose_seasons.png")
if cspd is not None:
    print("  fig/IA4_current_rose_seasons.png")
print("Saved table: fig/IA4_mean_wind_current_stats.csv")

**Q: What are the mean conditions (e.g. velocity and direction)? (if you have time: do you observe any seasonal variability?)**

### IA.4  Interpretation of wind and current conditions


**Answer.** Winds are W-WNW year-round and strongest in winter (overall mean 7.22 m/s; DJF mean 8.67 m/s). Currents are weak (about 0.10 m/s) and dominantly tidal along a NE-SW axis, so the overall mean direction is not physically representative.


---


### Data basis and conventions


- Period analysed: 1994-2020.  
- Calms excluded in roses: wind $<0.5$ m/s, current $<0.05$ m/s.  
- Wind: **coming-from**; Current: **going-to**; $0^\circ=\text{N}$, clockwise.  
- Bins used: wind speeds $[0.5,2),[2,4),\ldots,\ge 16$ m/s; current speeds $[0.05,0.10),[0.10,0.15),[0.15,0.20),\ge 0.20$ m/s; wind sectors $22.5^\circ$; current sectors $10^\circ$.


---


### Summary statistics


**Overall means (1994-2020)**


| Variable | Mean speed | Mean direction |
|---|---:|---:|
| Wind (coming-from) | 7.22 m/s | 296.6 |
| Current (going-to) | 0.096 m/s | 285.2* |


\*For reversing tidal currents, a single circular mean direction is not physically informative.


**Seasonal means**


| Season | Wind speed (m/s) | Wind dir (deg) | Current speed (m/s) | Current dir (deg)* |
|---|---:|---:|---:|---:|
| DJF | 8.67 | 266.3 | 0.096 | 211.3 |
| MAM | 6.95 | 321.1 | 0.096 | 28.3 |
| JJA | 6.00 | 302.6 | 0.096 | 228.5 |
| SON | 7.31 | 287.5 | 0.096 | 19.0 |


\*Seasonal current means are shown for completeness but the roses indicate a bidirectional tidal regime.


---


### Wind climate: what the roses show


- **Directionality.** Dominant **W-WNW** approach. Seasonal veer is modest: closer to **W** in winter (DJF $\sim266^\circ$), rotating towards **WNW** in spring and summer (MAM-JJA $\sim321^\circ$ to $\sim303^\circ$), then easing back in autumn (SON $\sim288^\circ$).
- **Intensity.** Clear seasonal cycle: **DJF > SON > MAM > JJA** by mean speed. The full-period rose shows most occurrences in the **4-12 m/s** bands, with winter contributing the higher **8-14 m/s** fractions.
- **Variability.** Sectoral spread remains broader in MAM/SON than in DJF/JJA, consistent with synoptic variability during shoulder seasons.


---


### Current climate: what the roses show


- **Regime.** Two narrow, opposing **going-to** headings dominate, approximately **NE (30^\circ)** and **SW (210^\circ)**, confirming a reversing tidal signal.
- **Magnitudes.** Most occurrences lie **below 0.30 m/s**; the highest occupied bin is typically **0.15-0.20 m/s**, with scarce excursions beyond. The mean speed of **0.096 m/s** reflects the prevalence of weak flows.
- **Seasonality.** The **axis does not shift** with season, and the speed distribution changes little between DJF and JJA in the roses, highlighting tide-dominated currents with weak seasonal modulation.


---


### Site context notes


- The study area is **south of Lorient (southern Brittany)**, exposed to the **open North Atlantic**. The prevailing **W-WNW** winds align with the basin-scale westerlies and frequent winter cyclones.  
- The **NE-SW** tidal current axis is consistent with coastal geometry that channels reversing flows; without detailed bathymetry/harmonic analysis, treat this as a qualitative inference from the roses.


---


### Caveats and suggested refinements


- **Currents:** Do not use circular means for direction. Prefer a **principal axis** metric and report the **two modal headings** with their shares.  
- **Quantiles:** Add $P_{50}$, $P_{90}$, $P_{95}$ for wind and current speeds to complement the means (e.g., $P_{90}$ wind speed $\approx$ x m/s).  
- **Calms:** Roses exclude calms by design; if calm frequency is relevant to operations, report the calm fraction separately.  
- **Next step:** If needed for design or logistics, compute a tidal ellipse or principal-component axis for the currents, and provide **hour-of-tide** roses to expose phase dependence.



---
### I.A.5. Comparison to Wave Buoy Measurements

We can validate the ResourceCode hindcast data against observations from a nearby wave buoy, for example, from the Candhis website.






**Q: Comparing the hindcast data from ResourceCode to the observations during the time period with overlapping data, what is the RMSD in the wave height, period, and direction between the observations and simulations?**

In [ ]:
# --- IA.5 — Candhis buoy validation (Belle-Île 05602, year 2020) ---
# Goal: RMSD between ResourceCode hindcast and Candhis observations for Hs, Tm02, and mean direction.
# Notes:
# - Candhis direction: use THETAM (mean, coming-from, true north, clockwise).
# - ResourceCode wave direction assumed coming-from in `data['dir']`. If not, set RC_IS_COMING_FROM=False.
# - Time matching: nearest within ±30 min.
# - QC: drop sentinels (999.*), non-numeric, |SKEW|>0.3, KURT>5, and (if present) non-valid QUALITE.

# ---------- Config ----------
RC_IS_COMING_FROM = True          # set to False if your ResourceCode 'dir' is going-to
TOL = pd.Timedelta("30min")       # matching tolerance
CSV_PATHS = [
    Path("data/Candhis_05602_2020_arch.csv"),     # preferred relative path
    Path("/mnt/data/Candhis_05602_2020_arch.csv") # fallback (chat attachment)
]

# ---------- Utilities ----------
def _first_existing(paths):
    for p in paths:
        if p.exists():
            return p
    raise FileNotFoundError("Candhis 2020 CSV not found at any known path.")

def circ_diff_deg(a_deg, b_deg):
    """Minimal angular difference a-b in degrees in [-180, 180)."""
    d = (a_deg - b_deg + 180.0) % 360.0 - 180.0
    return d

def rmse(a, b):
    a = np.asarray(a, float)
    b = np.asarray(b, float)
    m = np.isfinite(a) & np.isfinite(b)
    if m.sum() == 0:
        return np.nan, 0
    return float(np.sqrt(np.mean((a[m] - b[m])**2))), int(m.sum())

def circ_rmse_deg(a_deg, b_deg):
    """RMS of circular differences in degrees."""
    d = circ_diff_deg(np.asarray(a_deg, float), np.asarray(b_deg, float))
    m = np.isfinite(d)
    if m.sum() == 0:
        return np.nan, 0
    return float(np.sqrt(np.mean(d[m]**2))), int(m.sum())

# ---------- Load Candhis 2020 ----------
candhis_csv = _first_existing(CSV_PATHS)

cand = pd.read_csv(
    candhis_csv,
    sep=";", engine="python",
    parse_dates=["DateHeure"]
)

# Keep only fields needed + QC fields
need_cols = ["DateHeure", "HM0", "T02", "THETAM", "SKEW", "KURT", "QUALITE"]
for c in need_cols:
    if c not in cand.columns:
        # QUALITE may be absent; others must exist
        if c == "QUALITE":
            cand["QUALITE"] = np.nan
        else:
            raise KeyError(f"Missing '{c}' in Candhis CSV.")

# Convert to numeric and mark sentinels (e.g., 999.*) as NaN
num_cols = ["HM0", "T02", "THETAM", "SKEW", "KURT"]
for c in num_cols:
    cand[c] = pd.to_numeric(cand[c], errors="coerce")
    # Drop obvious sentinels like 999, 999.999, etc.
    cand.loc[cand[c] >= 999.0, c] = np.nan

# QC filters
qc = pd.Series(True, index=cand.index)

# 1) Required numeric fields present
qc &= cand["HM0"].notna() & cand["T02"].notna() & cand["THETAM"].notna()

# 2) Physical sanity
qc &= (cand["HM0"] > 0) & (cand["T02"] > 0)

# 3) Distribution checks
#    Drop timestamps with |SKEW| > 0.3 or KURT > 5 (when available)
skew_ok = cand["SKEW"].abs() <= 0.3
kurt_ok = cand["KURT"] <= 5.0
# If SKEW/KURT missing at a row, do not auto-drop for that reason
skew_ok = skew_ok | cand["SKEW"].isna()
kurt_ok = kurt_ok | cand["KURT"].isna()
qc &= skew_ok & kurt_ok

# 4) QUALITE (if present): keep only rows with non-empty labels considered valid
#    Many archive files have QUALITE empty; when populated, typical "good" tags include letters.
if cand["QUALITE"].notna().any():
    # Keep non-empty, non-null strings that are not "M", "NA", or "ERR"
    q = cand["QUALITE"].astype(str).str.strip().str.upper()
    good = ~(q.isna() | (q == "") | q.isin({"M", "NA", "ERR"}))
    qc &= good

cand_qc = cand.loc[qc, ["DateHeure", "HM0", "T02", "THETAM"]].copy()
cand_qc = cand_qc.rename(columns={"DateHeure": "time", "HM0": "hs_obs", "T02": "Tm02_obs", "THETAM": "dir_obs_deg"})
cand_qc["time"] = pd.to_datetime(cand_qc["time"], utc=True)  # TU = UTC
cand_qc = cand_qc.set_index("time").sort_index()

# Restrict strictly to 2020 UTC
start_utc = pd.Timestamp("2020-01-01 00:00:00", tz="UTC")
end_utc   = pd.Timestamp("2020-12-31 23:59:59", tz="UTC")
cand_qc = cand_qc.loc[start_utc:end_utc]

print(f"Candhis 2020 after QC: {len(cand_qc)} rows")

# ---------- Prepare ResourceCode subset for 2020 ----------
# Assumes your earlier cells defined `data` with columns: 'hs', 'Tm02', 'dir' and a DatetimeIndex in UTC or naive-UTC.
if not {"hs", "Tm02", "dir"}.issubset(data.columns):
    raise KeyError("ResourceCode `data` must include ['hs','Tm02','dir'].")

rc = data[["hs", "Tm02", "dir"]].copy()

# Ensure datetime is tz-aware UTC to compare with Candhis TU
if rc.index.tz is None:
    rc.index = rc.index.tz_localize("UTC")

rc_2020 = rc.loc[start_utc:end_utc].copy()
rc_2020 = rc_2020.sort_index()

# Direction convention alignment
if RC_IS_COMING_FROM:
    rc_2020["dir_from_deg"] = rc_2020["dir"] % 360.0
else:
    # Convert going-to -> coming-from to match THETAM
    rc_2020["dir_from_deg"] = (rc_2020["dir"] + 180.0) % 360.0

rc_2020 = rc_2020.rename(columns={"hs": "hs_mod", "Tm02": "Tm02_mod"})

# Time matching (nearest within ±30 min)
left  = cand_qc.reset_index().rename(columns={"time": "t_obs"})
right = rc_2020.reset_index().rename(columns={"index": "t_mod"})

pairs = pd.merge_asof(
    left.sort_values("t_obs"),
    right.sort_values("t_mod"),
    left_on="t_obs",
    right_on="t_mod",
    tolerance=TOL,
    direction="nearest",
)

# Drop non-matches (NaN where no model within tolerance)
pairs = pairs.dropna(subset=["hs_mod", "Tm02_mod", "dir_from_deg"])

# ---------- Metrics ----------
hs_rmse, n_hs = rmse(pairs["hs_mod"], pairs["hs_obs"])
t02_rmse, n_t = rmse(pairs["Tm02_mod"], pairs["Tm02_obs"])
dir_rmse, n_d = circ_rmse_deg(pairs["dir_from_deg"], pairs["dir_obs_deg"])

summary = pd.DataFrame({
    "metric": ["RMSD_Hs (m)", "RMSD_Tm02 (s)", "RMSD_Dir_THETAM (deg)"],
    "value": [hs_rmse, t02_rmse, dir_rmse],
    "N_pairs": [n_hs, n_t, n_d]
})

print("\nIA.5 — Candhis vs ResourceCode (2020, Belle-Île 05602)")
print(summary.to_string(index=False, float_format=lambda v: f"{v:.3f}"))

# Save outputs
out_dir = Path("fig")
out_dir.mkdir(exist_ok=True)
summary.to_csv(out_dir / "IA5_rmsd_2020_candhis05602.csv", index=False)

# Optional quick-look plots
fig, ax = plt.subplots(3, 1, figsize=(12, 9), sharex=True)
ax[0].plot(pairs["t_obs"], pairs["hs_obs"], label="Buoy HM0", lw=0.9)
ax[0].plot(pairs["t_obs"], pairs["hs_mod"], label="Model Hs", lw=0.9)
ax[0].set_ylabel("Hs (m)")
ax[0].grid(alpha=0.3)
ax[0].legend()

ax[1].plot(pairs["t_obs"], pairs["Tm02_obs"], label="Buoy T02", lw=0.9)
ax[1].plot(pairs["t_obs"], pairs["Tm02_mod"], label="Model Tm02", lw=0.9)
ax[1].set_ylabel("Tm02 (s)")
ax[1].grid(alpha=0.3)
ax[1].legend()

# Direction as coming-from; plot circularly unwrapped about buoy direction to visualise differences
d_err = circ_diff_deg(pairs["dir_from_deg"], pairs["dir_obs_deg"])
ax[2].plot(pairs["t_obs"], d_err, lw=0.8)
ax[2].axhline(0, color="k", lw=0.8)
ax[2].set_ylabel("Dir error (deg)")
ax[2].set_xlabel("2020 UTC")
ax[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(out_dir / "IA5_timeseries_2020.png", dpi=200, bbox_inches="tight")
plt.show()

# Scatter diagnostics (optional)
fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))
axes[0].scatter(pairs["hs_obs"], pairs["hs_mod"], s=4, alpha=0.5)
mx = np.nanmax([pairs["hs_obs"].max(), pairs["hs_mod"].max()])
axes[0].plot([0, mx], [0, mx], 'k:', lw=1)
axes[0].set_xlabel("Buoy HM0 (m)")
axes[0].set_ylabel("Model Hs (m)")
axes[0].set_title(f"RMSD={hs_rmse:.2f} m")

axes[1].scatter(pairs["Tm02_obs"], pairs["Tm02_mod"], s=4, alpha=0.5)
mx = np.nanmax([pairs["Tm02_obs"].max(), pairs["Tm02_mod"].max()])
axes[1].plot([0, mx], [0, mx], 'k:', lw=1)
axes[1].set_xlabel("Buoy T02 (s)")
axes[1].set_ylabel("Model Tm02 (s)")
axes[1].set_title(f"RMSD={t02_rmse:.2f} s")

axes[2].hist(d_err, bins=72)
axes[2].set_xlabel("Dir error (deg, THETAM)")
axes[2].set_ylabel("Count")
axes[2].set_title(f"circ-RMSD={dir_rmse:.1f}°")

plt.tight_layout()
plt.savefig(out_dir / "IA5_scatter_2020.png", dpi=200, bbox_inches="tight")
plt.show()

### IA.5 — Buoy–model comparison (Belle-Île 05602 vs ResourceCode), year 2020

**RMSD results** (±30 min pairing; QC applied; direction uses Candhis **THETAM**, coming-from):
- $H_s$: **0.39 m**  
- $T_{m02}$: **0.84 s**  
- Mean direction: **21.2°** (circular RMSD)  
Pairs used: **15 947** half-hours.

---

#### Interpretation

**Wave height.** Agreement is good over the 0–8 m range. Most error accrues during storms, as expected when model and buoy are not co-located.

**Period.** Consistent overall. The scatter indicates a mild mismatch at the longest periods.

**Direction.** A 21° circular RMSD is reasonable near a coast with mixed sea–swell and turning fronts. Spikes coincide with weak seas or multi-modal spectra where mean direction is less stable.

---

#### Spatial representativeness and depth effects

- **Positions.** Buoy: $(47.3111^\circ,\,-3.3111^\circ)$ at **45 m** depth.  
  Model point: $(47.3236^\circ,\,-3.5522^\circ)$ at **93.32 m** depth.  
  Horizontal offset ≈ **18 km**.

- **Depth regime difference.** Using the deep-water proxy $T_\text{deep}\approx 4\sqrt{h/g}$:
  - At **45 m**: $T_\text{deep}\approx 8.6$ s. Longer swell can enter transitional depth, with refraction and depth-induced transformation before reaching the buoy.
  - At **93.32 m**: $T_\text{deep}\approx 12.3$ s. The offshore model point remains deep for a larger share of the spectrum.

- **Implications.** Between 93 m and 45 m the wave field rotates and re-distributes energy. This raises direction error and can alter spectral shape enough to shift $T_{m02}$ slightly. Height differences during energetic events also reflect unmodelled small-scale sheltering by Belle-Île and local bathymetry not captured by the point-to-point comparison.

---

#### February–March 2020 gap

The ~**1-month** hole is from the **observations**, not the model. After QC (sentinels removed; $|SKEW|\le 0.3$; KURT $\le 5$; QUALITE filter when present), many half-hours in late winter drop out. Typical causes are buoy maintenance, telemetry loss, or spectra failing distribution checks. The line plots bridge over missing points; no data were used there.

---

#### Answer to the study question

For the 2020 overlap, the RMSD between ResourceCode and the Candhis Belle-Île buoy is **0.39 m** for $H_s$, **0.84 s** for $T_{m02}$, and **21.2°** for mean direction (THETAM, circular RMSD). Given the **18 km** separation and **93 m vs 45 m** depths, these errors are consistent with expected coastal transformation between the offshore model point and the nearer-shore buoy.

## I.B. Estimating the extreme wave conditions

### Block Maxima (BM) Method

**Approach:**  
This approach involves dividing the long-term time series of significant wave heights into non-overlapping blocks of equal duration, typically one year.

**Process:**  
The single highest significant wave height (Hₘ₀) is taken from each block (e.g., the annual maximum).

**Distribution:**  
These maximum values are then fitted to a *Generalized Extreme Value (GEV)* distribution.

**Outcome:**  
This model allows you to estimate the wave height corresponding to a specific return period, such as the 1-year or 50-year storm.

In [ ]:
# Step 1: Create EVA object
model_bm = EVA(data.hs)

# Step 2: Extract extremes
model_bm.get_extremes(
    method="BM",
    block_size="365.2425D",
    extremes_type="high",
    errors="raise",
    min_last_block=0.9
)


Lets check the directions of the extreme heights

In [ ]:
# lets find the directions of the extremes from the data
extremes_indices = model_bm.extremes.index
data_extreme_directions = data.loc[extremes_indices, "dir"]
data_extreme_tm02 = data.loc[extremes_indices, "Tm02"]
data_extreme_wspd = data.loc[extremes_indices, "wspd"]


In [ ]:
extremes_df = (
    pd.DataFrame({'timestamp': extremes_indices, 'hs': model.extremes.values})
    .merge(
        data[['dir','wspd','Tm02']].reset_index().rename(columns={'index':'timestamp'}),
        on='timestamp',
        how='left'
    )

)

In [ ]:
extremes_df.head()

In [ ]:
# extremes
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Plot 1: Time series with extremes highlighted
ax1.plot(data.index, data.hs, 'b-', alpha=0.3, linewidth=0.5, label='Full time series')
ax1.scatter(model.extremes.index, model.extremes.values, color='red', s=20, alpha=0.8, label='Extremes (Block Maxima)')
ax1.set_xlabel('Date')
ax1.set_ylabel('Significant Wave Height (m)')
ax1.set_title('Time Series with One-year Block Extremes')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Histogram of extremes
ax2.hist(model.extremes.values, bins=15, alpha=0.7, color='red', edgecolor='black')
ax2.set_xlabel('Extreme Wave Height (m)')
ax2.set_ylabel('Frequency')
ax2.set_title('Distribution of Block Maxima Extremes')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print some statistics about the extremes
print(f"Number of extremes: {len(model.extremes)}")
print(f"Mean extreme value: {model.extremes.mean():.3f} m")
print(f"Maximum extreme value: {model.extremes.max():.3f} m")
print(f"Minimum extreme value: {model.extremes.min():.3f} m")
print(f"Standard deviation: {model.extremes.std():.3f} m")

In [ ]:
# More detailed manual plotting of extremes
fig, ax = plt.subplots(1, 1, figsize=(12, 4))

# Plot the full time series
ax.plot(data.index, data.hs, 'b-', alpha=0.2, linewidth=0.3, label='Full time series')

# Highlight extremes with different colors based on magnitude
extremes = model.extremes
colors = plt.cm.Reds(np.linspace(0.3, 1, len(extremes)))
scatter = ax.scatter(extremes.index, extremes.values, c=extremes.values, 
                    cmap='Reds', s=30, alpha=0.8, edgecolors='black', linewidth=0.5)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Wave Height (m)')

# Customize the plot
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Significant Wave Height (m)', fontsize=12)
ax.set_title('Block Maxima Extremes Over Time\n(Red dots show annual maximum wave heights)', fontsize=14)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# lets execute GEV fit on the extremes
model_bm.fit_model()


In [ ]:
# Calculate return values for 1-year and 50-year period
return_period = [1.01,1.1, 1.2, 1.3, 1.4,1.5, 2, 5, 10, 25, 50]
summary_bm = model_bm.get_summary(return_period=return_period, alpha=0.95)
print(summary_bm)


In [ ]:
# Plot with correct empirical return periods
fig, ax = plt.subplots(figsize=(10, 6))

# Get correct empirical return periods
sorted_extremes = np.sort(model.extremes.values)[::-1]  # Sort descending
n_extremes = len(sorted_extremes)

# Correct empirical return period calculation
# For block maxima, empirical return period = (n + 1) / (rank)
empirical_return_periods = (n_extremes + 1) / np.arange(1, n_extremes + 1)

# Plot your observed data with correct empirical return periods
ax.semilogx(empirical_return_periods, sorted_extremes, 'ko', markersize=4, alpha=0.7, label='Observed Extremes (Empirical)')

# Add fitted model for comparison
summary = model.get_summary([0, 1, 2, 5, 10, 25, 50, 100], return_period_size="365.2425D", alpha=0.90)
ax.semilogx(summary.index, summary['return value'], 'r-', linewidth=2, label='Fitted Model')

# Add confidence intervals
ax.semilogx(summary.index, summary['upper ci'], 'r--', linewidth=1, alpha=0.7, label='Upper 90% CI')
ax.semilogx(summary.index, summary['lower ci'], 'r--', linewidth=1, alpha=0.7, label='Lower 90% CI')

# Fill between confidence intervals
ax.fill_between(summary.index, summary['lower ci'], summary['upper ci'], alpha=0.2, color='red')

# Highlight 50-year point
ax.semilogx(50, summary.loc[50, 'return value'], 'ro', markersize=8, label='50-year Return Period')

ax.set_xlabel('Return Period (years)')
ax.set_ylabel('Significant Wave Height (m)')
ax.set_title('Empirical vs Fitted Return Values')
ax.grid(True, alpha=0.3)
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Plot with extended model to 1 year
fig, ax = plt.subplots(figsize=(10, 6))

# Get correct empirical return periods
sorted_extremes = np.sort(model.extremes.values)[::-1]  # Sort descending
n_extremes = len(sorted_extremes)

# Correct empirical return period calculation
empirical_return_periods = (n_extremes + 1) / np.arange(1, n_extremes + 1)

# Plot your observed data with correct empirical return periods
ax.semilogx(empirical_return_periods, sorted_extremes, 'ko', markersize=4, alpha=0.7, label='Observed Extremes (Empirical)')

# Add fitted model with extended range to 1 year
summary = model.get_summary([1, 1.5, 2, 3, 5, 10, 25, 50, 100], return_period_size="365.2425D", alpha=0.90)
ax.semilogx(summary.index, summary['return value'], 'r-', linewidth=2, label='Fitted Model')

# Add confidence intervals
ax.semilogx(summary.index, summary['upper ci'], 'r--', linewidth=1, alpha=0.7, label='Upper 90% CI')
ax.semilogx(summary.index, summary['lower ci'], 'r--', linewidth=1, alpha=0.7, label='Lower 90% CI')

# Fill between confidence intervals
ax.fill_between(summary.index, summary['lower ci'], summary['upper ci'], alpha=0.2, color='red')

# Highlight 50-year point
ax.semilogx(50, summary.loc[50, 'return value'], 'ro', markersize=8, label='50-year Return Period')

ax.set_xlabel('Return Period (years)')
ax.set_ylabel('Significant Wave Height (m)')
ax.grid(True, alpha=0.3)
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# define the threshold for the POT method
# an empricial threshold is used here based on the data's return period
threshold = 2.5  # meters
print(f'Threshold: {threshold} m')


### Modelling univariate time series: Block maxima + GEVD (Generalized Extreme Value Distribution)

We show as an example here a **BM** (block maxima) model fitted to the $H_s$ time series. In this approach, the maximum value is identified within a "block" or fixed period in time, and then a GEVP distribution is fit to the data to estimate the return values.  

The same plot can readily be obtained for the other sea-state parameters.

In [ ]:
summary_pot = model_pot.get_summary(
    return_period=[1, 2, 5, 10, 25, 50],
    return_period_size="365.2425D",
    alpha=0.95
)
print(summary_pot)


After loading the data, apply the block method approach with a block size of 1 year (365.2425 days), where each data block must be at least 90% full to take into account in the analysis.

In [ ]:
print(summary_bm)


In [ ]:
print(summary_pot)


In [ ]:
model.plot_extremes()

In [ ]:
model.fit_model()

The parameter alpha specifies the confidence limits (default = 0.95).

In [ ]:
model_pot.plot_diagnostic(alpha=0.95)
plt.savefig('graphs/diagnostic_plot_pot.png', dpi=200, bbox_inches='tight')
plt.show()


The parameter n_samples indicates the number of bootstrap samples used to estimate the confidence bounds.

In [ ]:
summary = model.get_summary(
        return_period=[1, 2, 5, 10, 25, 50, 100, 250, 500, 1000],
        alpha=0.95,
        n_samples=1000,
    )
print(summary)

### Modelling univariate time series: Peaks over threshold (POT) + GPD (Generalized Pareto Distribution)

We show as example here a **POT** (peaks over threshold) model fitted to the $H_s$ time series. This analysis first finds values over a specified threshold and then declusters these values using a predefined clustering distance, and finally finds the maximum value within each cluster. 

The same plot can readily be obtained for the other sea-state parameters.

We first can have a look at the quality of the fitted model, and to the corresponding return levels as a function of the selected wave height threshold. The parameters r and alpha specify the minimum time distance (duration) between adjacent clusters and the confidence limits (default = 0.95), respectively.

The shape and modified scale parameters define the Generalized Pareto Distribution, and they depend on the threshold value, but should be stable within a range of valid thresholds (e.g. less than ~3m here).

In [ ]:
plot_parameter_stability(ts=data.hs, r='72H', alpha=0.95)

The mean residual life plots the average excess value over a given threshold, and it should be approcimately linear above the threshold for which the GPD model is valid (e.g. <~3m)

In [ ]:
plot_mean_residual_life(data.hs)

The analysis is completed for both Hs and the wind speed, specifying a window of 72 hours and a quantile of 0.98 for determining the threshold to specify.

In [ ]:
quant=0.98
models = get_fitted_models(data[["hs","wspd"]],quantile=quant,r="72H")

In [ ]:
models

In [ ]:
models[0].plot_diagnostic(alpha=0.95)

In [ ]:
models[1].plot_diagnostic(alpha=0.95)

In [ ]:
pd.DataFrame(get_gpd_parameters(models),columns=["mu","sigma","xi"],index=["Hs","Wspd"])

In [ ]:
summary_Hs = models[0].get_summary(
    return_period=[1, 2, 5, 10, 25, 50, 100],
    alpha=0.95,
    n_samples=1000,
)
summary_Wspd = models[1].get_summary(
    return_period=[1, 2, 5, 10, 25, 50, 100],
    alpha=0.95,
    n_samples=1000,
)
print(summary_Hs)
print(summary_Wspd)